In [2]:
import json
import itertools
import zipfile
from datetime import date
from pathlib import Path

import cv2
import rasterio
import utm
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sentinelsat import SentinelAPI, read_geojson, geojson_to_wkt
from sentinelhub import pixel_to_utm, utm_to_pixel

In [3]:
# Using pathlib for handling paths which is convientent way. And set path to ../data folder from current context.
data_path = Path("../data/")

In [20]:
# Set your user and password.
with open("../secrets/sentinel_api_credentials.json", "r") as file:
    credentials = json.load(file)

api = SentinelAPI(credentials["username"], credentials["password"], "https://scihub.copernicus.eu/dhus")

shape = geojson_to_wkt(read_geojson(data_path / "cameroon.geojson"))

# Try to make the data be between 2 months because image may not be captured by satellite for small intervals.
images = api.query(
    shape,
    date=(date(2023, 1, 1), date(2023, 3, 1)),
    platformname="Sentinel-2",
    processinglevel = "Level-2A",
    cloudcoverpercentage=(0, 30)
)

images_df = api.to_dataframe(images)

In [21]:
images_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 34 entries, 386009ee-2a37-4b58-9f55-03f0d5cea946 to 39f7c5d8-2df1-4b4d-ae05-63641f364fa2
Data columns (total 42 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   title                        34 non-null     object        
 1   link                         34 non-null     object        
 2   link_alternative             34 non-null     object        
 3   link_icon                    34 non-null     object        
 4   summary                      34 non-null     object        
 5   ondemand                     34 non-null     object        
 6   generationdate               34 non-null     datetime64[ns]
 7   beginposition                34 non-null     datetime64[ns]
 8   endposition                  34 non-null     datetime64[ns]
 9   ingestiondate                34 non-null     datetime64[ns]
 10  orbitnumber                  34 non-null     int

In [22]:
images_df.head()

,title,link,link_alternative,link_icon,summary,ondemand,generationdate,beginposition,endposition,ingestiondate,...,s2datatakeid,producttype,platformidentifier,orbitdirection,platformserialidentifier,processinglevel,datastripidentifier,granuleidentifier,identifier,uuid
386009ee-2a37-4b58-9f55-03f0d5cea946,S2A_MSIL2A_20230220T093031_N0509_R136_T32NNL_2...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,"Date: 2023-02-20T09:30:31.025Z, Instrument: MS...",false,2023-02-20 14:34:08,2023-02-20 09:30:31.025,2023-02-20 09:30:31.025,2023-02-20 16:09:35.637,...,GS2A_20230220T093031_040030_N05.09,S2MSI2A,2015-028A,DESCENDING,Sentinel-2A,Level-2A,S2A_OPER_MSI_L2A_DS_2APS_20230220T143408_S2023...,S2A_OPER_MSI_L2A_TL_2APS_20230220T143408_A0400...,S2A_MSIL2A_20230220T093031_N0509_R136_T32NNL_2...,386009ee-2a37-4b58-9f55-03f0d5cea946
e3eea048-79fd-4f9d-8c3c-450998853276,S2A_MSIL2A_20230220T093031_N0509_R136_T32NNK_2...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,"Date: 2023-02-20T09:30:31.025Z, Instrument: MS...",false,2023-02-20 14:34:08,2023-02-20 09:30:31.025,2023-02-20 09:30:31.025,2023-02-20 16:09:24.695,...,GS2A_20230220T093031_040030_N05.09,S2MSI2A,2015-028A,DESCENDING,Sentinel-2A,Level-2A,S2A_OPER_MSI_L2A_DS_2APS_20230220T143408_S2023...,S2A_OPER_MSI_L2A_TL_2APS_20230220T143408_A0400...,S2A_MSIL2A_20230220T093031_N0509_R136_T32NNK_2...,e3eea048-79fd-4f9d-8c3c-450998853276
8daddb67-7ad8-4838-a460-dab8ab3a4dda,S2A_MSIL2A_20230220T093031_N0509_R136_T32NPL_2...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,"Date: 2023-02-20T09:30:31.025Z, Instrument: MS...",false,2023-02-20 14:34:08,2023-02-20 09:30:31.025,2023-02-20 09:30:31.025,2023-02-20 16:08:32.950,...,GS2A_20230220T093031_040030_N05.09,S2MSI2A,2015-028A,DESCENDING,Sentinel-2A,Level-2A,S2A_OPER_MSI_L2A_DS_2APS_20230220T143408_S2023...,S2A_OPER_MSI_L2A_TL_2APS_20230220T143408_A0400...,S2A_MSIL2A_20230220T093031_N0509_R136_T32NPL_2...,8daddb67-7ad8-4838-a460-dab8ab3a4dda
2153894d-6887-45bf-a30c-4edeed334f5e,S2A_MSIL2A_20230220T093031_N0509_R136_T32NQL_2...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,"Date: 2023-02-20T09:30:31.025Z, Instrument: MS...",false,2023-02-20 14:34:08,2023-02-20 09:30:31.025,2023-02-20 09:30:31.025,2023-02-20 16:06:24.627,...,GS2A_20230220T093031_040030_N05.09,S2MSI2A,2015-028A,DESCENDING,Sentinel-2A,Level-2A,S2A_OPER_MSI_L2A_DS_2APS_20230220T143408_S2023...,S2A_OPER_MSI_L2A_TL_2APS_20230220T143408_A0400...,S2A_MSIL2A_20230220T093031_N0509_R136_T32NQL_2...,2153894d-6887-45bf-a30c-4edeed334f5e
5dca32b8-24de-4a33-8bd8-f1acf34af6ca,S2A_MSIL2A_20230220T093031_N0509_R136_T32NRL_2...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,"Date: 2023-02-20T09:30:31.025Z, Instrument: MS...",false,2023-02-20 14:34:08,2023-02-20 09:30:31.025,2023-02-20 09:30:31.025,2023-02-20 16:02:50.587,...,GS2A_20230220T093031_040030_N05.09,S2MSI2A,2015-028A,DESCENDING,Sentinel-2A,Level-2A,S2A_OPER_MSI_L2A_DS_2APS_20230220T143408_S2023...,S2A_OPER_MSI_L2A_TL_2APS_20230220T143408_A0400...,S2A_MSIL2A_20230220T093031_N0509_R136_T32NRL_2...,5dca32b8-24de-4a33-8bd8-f1acf34af6ca


In [43]:
# Randomaly select a uuid.
uuids = images_df["uuid"].sample().iloc[0]
uuids

'818c53ea-c39d-44ce-8bae-adf27769576c'

In [44]:
details = api.download(uuids, directory_path=data_path)
details

MD5 checksumming:   0%|          | 0.00/1.16G [00:00<?, ?B/s]

{'id': '818c53ea-c39d-44ce-8bae-adf27769576c',
 'title': 'S2B_MSIL2A_20230106T093259_N0509_R136_T32NQL_20230106T122128',
 'size': 1162740424,
 'md5': 'f420e417fae5a87ef685627212a7b3b2',
 'date': datetime.datetime(2023, 1, 6, 9, 32, 59, 24000),
 'footprint': 'POLYGON((10.80472775247355 5.425531020312141,11.795064553781264 5.42176328831644,11.790920736919428 4.429629774710796,10.80204968546757 4.432705100697929,10.80472775247355 5.425531020312141))',
 'url': "https://scihub.copernicus.eu/dhus/odata/v1/Products('818c53ea-c39d-44ce-8bae-adf27769576c')/$value",
 'Online': True,
 'Creation Date': datetime.datetime(2023, 1, 6, 15, 5, 4, 431000),
 'Ingestion Date': datetime.datetime(2023, 1, 6, 15, 4, 38, 318000),
 'quicklook_url': "https://scihub.copernicus.eu/dhus/odata/v1/Products('818c53ea-c39d-44ce-8bae-adf27769576c')/Products('Quicklook')/$value",
 'path': '../data/S2B_MSIL2A_20230106T093259_N0509_R136_T32NQL_20230106T122128.zip',
 'downloaded_bytes': 1162740424}

In [45]:
# Extract the zip file and delete it.
if Path(details["path"]).exists():
    with zipfile.ZipFile(details["path"], 'r') as zip_ref:
        print("Extracting and deleting zipfile.")
        zip_ref.extractall(data_path)

    Path(details["path"]).unlink(missing_ok=True)
else:
    print("Zip file does not exist.")

Extracting and deleting zipfile.


In [ ]:
## Still working on it.

def get_band(image_folder, band, resolution=10):

    # subfolder = [f for f in os.listdir(image_folder + "/GRANULE") if f[0]  == "L"][0]
    subfolder = [f for f in (Path(image_folder) / "GRANULE").iterdir() if f.name.startswith("L")][0]

    # image_folder_path  = image_folder / "GRANULE" / subfolder / "IMG_DATA" / f"R{resolution}m"

    # image_files = [im for im in os.listdir(image_folder_path) if im[-4:] == ".jp2"]
    # selected_file = [im for im in image_files if im.split("_")[2] == band][0]
    
    # with rasterio.open(f"{image_folder_path}/{selected_file}") as infile:
    #     img = infile.read(1)
            
    # return img

In [7]:
data_1_folder_path = data_path / "S2B_MSIL2A_20230106T093259_N0509_R136_T32NQL_20230106T122128.SAFE"

sub_folder_path = [f for f in (data_1_folder_path / "GRANULE").iterdir() if f.name.startswith("L")][0]
sub_folder_path

PosixPath('../data/S2B_MSIL2A_20230106T093259_N0509_R136_T32NQL_20230106T122128.SAFE/GRANULE/L2A_T32NQL_A030478_20230106T094402')

In [8]:
image_folder_path  = sub_folder_path / "IMG_DATA" / f"R{10}m"
image_folder_path

PosixPath('../data/S2B_MSIL2A_20230106T093259_N0509_R136_T32NQL_20230106T122128.SAFE/GRANULE/L2A_T32NQL_A030478_20230106T094402/IMG_DATA/R10m')

In [9]:
# check the image path's in image_folder_path of R10m
for path in image_folder_path.iterdir():
    print(path)

../data/S2B_MSIL2A_20230106T093259_N0509_R136_T32NQL_20230106T122128.SAFE/GRANULE/L2A_T32NQL_A030478_20230106T094402/IMG_DATA/R10m/T32NQL_20230106T093259_AOT_10m.jp2
../data/S2B_MSIL2A_20230106T093259_N0509_R136_T32NQL_20230106T122128.SAFE/GRANULE/L2A_T32NQL_A030478_20230106T094402/IMG_DATA/R10m/T32NQL_20230106T093259_WVP_10m.jp2
../data/S2B_MSIL2A_20230106T093259_N0509_R136_T32NQL_20230106T122128.SAFE/GRANULE/L2A_T32NQL_A030478_20230106T094402/IMG_DATA/R10m/T32NQL_20230106T093259_B03_10m.jp2
../data/S2B_MSIL2A_20230106T093259_N0509_R136_T32NQL_20230106T122128.SAFE/GRANULE/L2A_T32NQL_A030478_20230106T094402/IMG_DATA/R10m/T32NQL_20230106T093259_B04_10m.jp2
../data/S2B_MSIL2A_20230106T093259_N0509_R136_T32NQL_20230106T122128.SAFE/GRANULE/L2A_T32NQL_A030478_20230106T094402/IMG_DATA/R10m/T32NQL_20230106T093259_TCI_10m.jp2
../data/S2B_MSIL2A_20230106T093259_N0509_R136_T32NQL_20230106T122128.SAFE/GRANULE/L2A_T32NQL_A030478_20230106T094402/IMG_DATA/R10m/T32NQL_20230106T093259_B02_10m.jp2
../d